In [2]:
import os.path
import sys
sys.path.append('../')

from procedures import parse_missing
from importlib import reload
from utils import load_code_graph


In [15]:
import json
import pymongo
from utils.config import *
client = pymongo.MongoClient(mongo_url, mongo_port)
project_name = 'batch-processing-gateway'
database = client[project_name]
playground = playground_dir
with open(os.path.join(playground, project_name, "meta.json"), 'r') as file:
    meta = json.load(file)

# Notebook to generate slice patch

In [16]:
reload(load_code_graph)
reload(parse_missing)
for method_name in database.list_collection_names():
    if method_name not in meta['method_name_to_idx']:
        print(f"{method_name} not found in meta. You manually delete it?")
        continue
    log_dir = os.path.join(playground_dir, project_name, 'methods_no_slice', meta['method_name_to_idx'][method_name])
    if len(os.listdir(os.path.join(log_dir, 'full_report'))) == 0:
        print(f"No executable code from {log_dir}")
        continue
    print(f"Parsing missing from {log_dir}")
    _ = parse_missing.parse_missing(log_dir, database[method_name])

Parsing missing from /root/experiments/playground/batch-processing-gateway/methods_no_slice/method_0
Parsing missing from /root/experiments/playground/batch-processing-gateway/methods_no_slice/method_1
No executable code from /root/experiments/playground/batch-processing-gateway/methods_no_slice/method_2
Parsing missing from /root/experiments/playground/batch-processing-gateway/methods_no_slice/method_3
Parsing missing from /root/experiments/playground/batch-processing-gateway/methods_no_slice/method_4


Parsing missing from /root/experiments/playground/batch-processing-gateway/methods_no_slice/method_5
No executable code from /root/experiments/playground/batch-processing-gateway/methods_no_slice/method_6


In [17]:
# export all file lines for the Java slicer to read
slicing = list([])
for method_name in database.list_collection_names():
    if method_name not in meta['method_name_to_idx']:
        print(f"{method_name} not found in meta. You manually delete it?")
        continue
    log_dir = os.path.join(playground_dir, project_name, 'methods_no_slice', meta['method_name_to_idx'][method_name])
    slice_path = os.path.join(log_dir, 'slice_fixing', 'missing_slices.jsonl')
    if os.path.exists(slice_path):
        with open(slice_path, 'r') as file:
            content = file.read()
            if content != "":
                slicing.append(os.path.abspath(slice_path))
    with open(os.path.join(playground_dir, project_name, "slicing_tasks.txt"), 'w') as file:
        file.write('\n'.join(slicing))

In [18]:
# write scope file name
from utils import test_runner
modules = [os.path.dirname(p) for p in test_runner.parse_root_pom(meta['put_path'])]
bin_dirs = [os.path.join(module, 'target/classes') for module in modules if os.path.exists(os.path.join(module, 'target/classes'))]
with open(os.path.join(playground_dir, project_name, 'scope_wala.txt'), 'w') as file:
    file.write('Primordial,Java,stdlib,base\nPrimordial,Java,jarFile,primordial.jar.model\n')
    file.write('\n'.join([f"Application,Java,binaryDir,{bin_dir}" for bin_dir in bin_dirs]))